In [ ]:
import glob
import pandas as pd
import os

import pyspark
import warnings
warnings.filterwarnings(action='ignore')


from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .getOrCreate()

import pyspark.sql.functions as F

In [ ]:
os.getcwd()

In [ ]:
df = spark.read.parquet('output/full_data.parquet/')

In [ ]:
df.count()

In [ ]:
#Nulls
df.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in ['PRCP','SNOW']]).show()

In [ ]:
#Zeros
df.select([F.count(F.when(df[c] == 0, c)).alias(c) for c in ['PRCP','SNOW']]).show()

In [ ]:
clean_df = df.dropna(subset=['PRCP'])

In [ ]:
clean_df.count()

In [ ]:
clean_df = clean_df.withColumn("FLAG_PRCP_NON_ZERO", F.when(clean_df['PRCP']!=0,1).otherwise(0))
clean_df = clean_df.withColumn("YEAR", clean_df.YEARMONTH.substr(0,4)).withColumn("MONTH", clean_df.YEARMONTH.substr(5,2)).withColumn("DAY", clean_df.YEARMONTH.substr(7,2))

In [ ]:
clean_df.show()

In [ ]:
# #STATION, MONTH, YEAR
# median_df = clean_df.groupBy("YEAR","MONTH","STATION","STATE_CITY_NAME").agg(F.sum("FLAG_PRCP_NON_ZERO").alias("SUM_NON_ZERO"))
# median_df.show()

In [ ]:
agg_df_y_m_prcp = clean_df.groupBy("YEAR","MONTH","STATE_CITY_NAME").agg(F.max("PRCP").alias("MAX_PRCP"), \
                                                                     F.min("PRCP").alias("MIN_PRCP"), \
                                                                     F.mean("PRCP").alias("MEAN_PRCP"), \
                                                                     F.percentile_approx("PRCP", 0.5).alias("MEDIAN_PRCP"), \
                                                                     F.mean("LONG").alias("MEAN_LONG"), \
                                                                     F.mean("LAT").alias("MEAN_LAT"))

In [ ]:
agg_df_y_prcp = clean_df.groupBy("YEAR","MONTH","STATE_CITY_NAME").agg(F.max("PRCP").alias("MAX_PRCP"), \
                                                                     F.min("PRCP").alias("MIN_PRCP"), \
                                                                     F.mean("PRCP").alias("MEAN_PRCP"), \
                                                                     F.percentile_approx("PRCP", 0.5).alias("MEDIAN_PRCP"), \
                                                                     F.mean("LONG").alias("MEAN_LONG"), \
                                                                     F.mean("LAT").alias("MEAN_LAT"))

In [ ]:
agg_df_y_m_prcp.write.parquet('output/agg_df_y_m_prcp.parquet')
agg_df_y_prcp.write.parquet('output/agg_df_y_prcp.parquet')